# Data Cleaning 101: Removing Duplicates and Fixing Errors (Titanic Dataset)

This notebook provides a concise, point-to-point guide on essential data cleaning techniques using the Titanic dataset. We will cover duplicate removal and common error fixing in Pandas.

## 1. Setup: Load the Dataset

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

# Load the Titanic dataset
df = sns.load_dataset('titanic')

print("Original DataFrame Info:")
df.info()
print("\nFirst 5 rows:")
df.head()

Original DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB

First 5 rows:


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## 2. Duplicate Detection and Removal

### 2.1. Spotting Duplicates with `df.duplicated()`

`df.duplicated()` returns a boolean Series indicating duplicate rows. `True` means the row is a duplicate of a previous one.

In [4]:
# Create a synthetic duplicate for demonstration
duplicate_row = df.iloc[0].copy()
df_with_dup = pd.concat([df, pd.DataFrame([duplicate_row])], ignore_index=True)

print("Number of duplicated rows (including synthetic):", df_with_dup.duplicated().sum())
print("\nFirst 10 duplicated rows (if any, showing both original and duplicate):", df_with_dup[df_with_dup.duplicated(keep=False)].sort_values(by=list(df_with_dup.columns)).head(10))

Number of duplicated rows (including synthetic): 108

First 10 duplicated rows (if any, showing both original and duplicate):      survived  pclass   sex   age  sibsp  parch     fare embarked   class  \
64          0       1  male   NaN      0      0  27.7208        C   First   
295         0       1  male   NaN      0      0  27.7208        C   First   
144         0       2  male  18.0      0      0  11.5000        S  Second   
757         0       2  male  18.0      0      0  11.5000        S  Second   
658         0       2  male  23.0      0      0  13.0000        S  Second   
733         0       2  male  23.0      0      0  13.0000        S  Second   
734         0       2  male  23.0      0      0  13.0000        S  Second   
134         0       2  male  25.0      0      0  13.0000        S  Second   
343         0       2  male  25.0      0      0  13.0000        S  Second   
666         0       2  male  25.0      0      0  13.0000        S  Second   

     who  adult_male deck 

C:\Users\Abdul Rehman\AppData\Local\Temp\ipykernel_14952\3137717905.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_with_dup = pd.concat([df, pd.DataFrame([duplicate_row])], ignore_index=True)


### 2.2. Removing Duplicates with `df.drop_duplicates()`

`df.drop_duplicates()` removes redundant rows, keeping the first occurrence by default.

In [5]:
df_cleaned_duplicates = df_with_dup.drop_duplicates()
print("DataFrame shape after dropping duplicates:", df_cleaned_duplicates.shape)

DataFrame shape after dropping duplicates: (784, 15)


### 2.3. Resetting Index Cleanly with `reset_index()`

After dropping rows, the index might be non-sequential. `reset_index(drop=True)` creates a clean, 0-based index.

In [7]:
df_final = df_cleaned_duplicates.reset_index(drop=True)
print("DataFrame head after resetting index:")
df_final.head()

DataFrame head after resetting index:


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## 3. Fixing Common Errors

### 3.1. Fixing String Errors with `str.lower()`, `str.strip()`, `str.replace()`

Use string methods for inconsistent casing, extra spaces, or specific text replacements.

In [9]:
# Introduce synthetic errors in 'Sex' column for demo
df_error_fix = df.copy()
df_error_fix.loc[df_error_fix["sex"] == "male", "sex"] = "Male "
df_error_fix.loc[df_error_fix["sex"] == "female", "sex"] = "FEMALE"
print("Unique values in \"sex\" before fixing:", df_error_fix["sex"].unique())

# Fix errors
df_error_fix["sex"] = df_error_fix["sex"].str.lower().str.strip()
print("Unique values in \"sex\" after fixing:", df_error_fix["sex"].unique())

Unique values in "sex" before fixing: ['Male ' 'FEMALE']
Unique values in "sex" after fixing: ['male' 'female']


### 3.2. Avoiding the `inplace=True` Trap (Prefer Reassignment)

Avoid `inplace=True` as it can lead to unexpected behavior. Always prefer reassignment for clarity and predictability.

In [12]:
# Preferred: Reassignment
df_reassigned = df.copy()
df_reassigned['age'] = df_reassigned['age'] * 2 # Example transformation
print("Original df head (Age unchanged):\n", df.head())
print("\nReassigned df_reassigned head (Age multiplied by 2):\n")
df_reassigned.head()

Original df head (Age unchanged):
    survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True  

Reassigned df_reassigned head (Age multiplied by 2):



,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,44.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,76.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,52.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,70.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,70.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Conclusion

Mastering these data cleaning techniques is crucial for any data professional. Clean data leads to reliable insights and robust models. Keep practicing!